## First merge the two datasets so they line up appropriately

In [1]:
import pandas as pd

# 1. Load each half-hour
feat = pd.read_parquet("order_book_feature.parquet")   # seconds 0–1799
tgt  = pd.read_parquet("order_book_target.parquet")    # seconds 1800–3599

# 2. Vertically concatenate them
full = pd.concat([feat, tgt], axis=0, ignore_index=True)

full.head()


,stock_id,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,8382,12,1.0,722.17,722.63,722.15,722.64,100,25,25,20
1,8382,12,2.0,722.18,722.88,722.17,722.98,25,10,66,100
2,8382,12,3.0,722.33,722.65,722.27,722.74,25,120,200,25
3,8382,12,4.0,722.68,722.98,722.48,723.00,25,100,25,1
4,8382,12,5.0,722.52,722.96,722.42,722.97,125,20,3,110


In [2]:
full.tail()

,stock_id,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
35557446,104919,1199,3595.0,362.73,362.74,362.72,362.75,200,1190,1200,1600
35557447,104919,1199,3596.0,362.68,362.70,362.67,362.71,800,200,1600,1400
35557448,104919,1199,3597.0,362.69,362.70,362.68,362.71,200,900,1400,1400
35557449,104919,1199,3598.0,362.72,362.73,362.71,362.74,200,1000,900,500
35557450,104919,1199,3599.0,362.80,362.81,362.79,362.82,200,300,600,300


In [3]:
full_sorted = full.sort_values(
    by=["stock_id", "time_id", "seconds_in_bucket"],
    ascending=[True,     True,      True]
).reset_index(drop=True)

# sanity-check the first few rows
full_sorted.tail()

,stock_id,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
35557446,104919,1199,3595.0,362.73,362.74,362.72,362.75,200,1190,1200,1600
35557447,104919,1199,3596.0,362.68,362.70,362.67,362.71,800,200,1600,1400
35557448,104919,1199,3597.0,362.69,362.70,362.68,362.71,200,900,1400,1400
35557449,104919,1199,3598.0,362.72,362.73,362.71,362.74,200,1000,900,500
35557450,104919,1199,3599.0,362.80,362.81,362.79,362.82,200,300,600,300


In [4]:
full_sorted = full_sorted[full_sorted["time_id"] >= 12].reset_index(drop=True)
full_sorted

,stock_id,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,8382,12,1.0,722.17,722.63,722.15,722.64,100,25,25,20
1,8382,12,2.0,722.18,722.88,722.17,722.98,25,10,66,100
2,8382,12,3.0,722.33,722.65,722.27,722.74,25,120,200,25
3,8382,12,4.0,722.68,722.98,722.48,723.00,25,100,25,1
4,8382,12,5.0,722.52,722.96,722.42,722.97,125,20,3,110
...,...,...,...,...,...,...,...,...,...,...,...
35363978,104919,1199,3595.0,362.73,362.74,362.72,362.75,200,1190,1200,1600
35363979,104919,1199,3596.0,362.68,362.70,362.67,362.71,800,200,1600,1400
35363980,104919,1199,3597.0,362.69,362.70,362.68,362.71,200,900,1400,1400
35363981,104919,1199,3598.0,362.72,362.73,362.71,362.74,200,1000,900,500


In [5]:

def pad_and_ffill(group):
    # 1) move seconds_in_bucket to the index
    grp = group.set_index('seconds_in_bucket')
    # 2) reindex so we have every second from 1 to 3599
    full_idx = range(1, 3600)
    grp = grp.reindex(full_idx)
    # 3) fill the grouping keys so they survive the reindex
    grp['stock_id'] = grp['stock_id'].ffill().bfill()
    grp['time_id' ] = grp['time_id' ].ffill().bfill()
    # 4) forward-fill all other columns
    grp = grp.ffill()
    # 5) reset index back into a column
    return grp.reset_index().rename(columns={'index':'seconds_in_bucket'})

# apply to each stock & time_id
full_seq = (
    full_sorted
    .groupby(['stock_id','time_id'], group_keys=False)
    .apply(pad_and_ffill)
    .reset_index(drop=True)
)

full_sorted = full_seq.sort_values(
    by=["stock_id", "time_id", "seconds_in_bucket"],
    ascending=[True,     True,      True]
).reset_index(drop=True)


/var/folders/g9/xf2qnfvn0wgfz305f2bqmzbr0000gn/T/ipykernel_29009/3360898526.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  full_sorted


In [6]:
full_sorted

,seconds_in_bucket,stock_id,time_id,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,1,8382.0,12.0,722.17,722.63,722.15,722.64,100.0,25.0,25.0,20.0
1,2,8382.0,12.0,722.18,722.88,722.17,722.98,25.0,10.0,66.0,100.0
2,3,8382.0,12.0,722.33,722.65,722.27,722.74,25.0,120.0,200.0,25.0
3,4,8382.0,12.0,722.68,722.98,722.48,723.00,25.0,100.0,25.0,1.0
4,5,8382.0,12.0,722.52,722.96,722.42,722.97,125.0,20.0,3.0,110.0
...,...,...,...,...,...,...,...,...,...,...,...
41460475,3595,104919.0,1199.0,362.73,362.74,362.72,362.75,200.0,1190.0,1200.0,1600.0
41460476,3596,104919.0,1199.0,362.68,362.70,362.67,362.71,800.0,200.0,1600.0,1400.0
41460477,3597,104919.0,1199.0,362.69,362.70,362.68,362.71,200.0,900.0,1400.0,1400.0
41460478,3598,104919.0,1199.0,362.72,362.73,362.71,362.74,200.0,1000.0,900.0,500.0


In [7]:
import numpy as np
import pandas as pd

# ─────────────────────────────────────────────────────────────────────────────
# 1. Start from your full_sorted DataFrame (with raw tick‐level columns still present)
# ─────────────────────────────────────────────────────────────────────────────
df = full_sorted.copy()

# ─────────────────────────────────────────────────────────────────────────────
# 2. Combine levels 1 & 2, then build all the features you listed
# ─────────────────────────────────────────────────────────────────────────────
# 2.1 price & returns
df['bid_price'] = df[['bid_price1','bid_price2']].mean(axis=1)
df['ask_price'] = df[['ask_price1','ask_price2']].mean(axis=1)
df['log_mid']   = np.log((df['bid_price'] + df['ask_price']) / 2)
df['log_ret']   = df.groupby(['stock_id','time_id'])['log_mid'].transform('diff')
df.dropna(subset=['log_ret'], inplace=True)

# 2.2 spread, depth, imbalance
df['spread']      = df['ask_price'] - df['bid_price']
df['depth']       = df[['bid_size1','bid_size2','ask_size1','ask_size2']].sum(axis=1)
df['imbalance']   = (
    (df['bid_size1']+df['bid_size2']) -
    (df['ask_size1']+df['ask_size2'])
) / (df['depth'] + 1e-9)
df['depth_ratio'] = (df['bid_size1']+df['bid_size2']) / (
    (df['ask_size1']+df['ask_size2']) + 1e-9
)

# 2.3 rolling‐window signals (5‐tick lookback)
df['rv_rolling']  = df.groupby(['stock_id','time_id'])['log_ret'] \
                       .transform(lambda x: x.rolling(5).std())
df['mom_rolling'] = df.groupby(['stock_id','time_id'])['log_ret'] \
                       .transform(lambda x: x.rolling(5).mean())

# ─────────────────────────────────────────────────────────────────────────────
# 3. Bucket‐aggregation function producing exactly your desired columns
# ─────────────────────────────────────────────────────────────────────────────
def aggregate_buckets(df, bucket_size):
    df_b = df.copy()
    df_b['bucket_start'] = (df_b['seconds_in_bucket'] // bucket_size) * bucket_size

    return (
        df_b
        .groupby(['stock_id','time_id','bucket_start'], as_index=False)
        .agg(
            log_ret_mean        = ('log_ret',    'mean'),
            log_ret_std         = ('log_ret',    'std'),
            log_ret_sum         = ('log_ret',    'sum'),
            abs_volatility      = ('log_ret',    lambda x: x.abs().sum()),
            realized_volatility = ('log_ret',    lambda x: np.sqrt((x**2).sum())),
            spread              = ('spread',    'mean'),
            imbalance           = ('imbalance', 'mean'),
            depth               = ('depth',     'sum'),
            depth_ratio         = ('depth_ratio','mean'),
            rv_rolling          = ('rv_rolling','last'),
            mom_rolling         = ('mom_rolling','last'),
        )
    )



In [8]:
df_30s = aggregate_buckets(df, 30)
print("30s buckets:", df_30s.shape)

30s buckets: (1382400, 14)


In [9]:
df_60s = aggregate_buckets(df, 60)
print("60s buckets:", df_60s.shape)

60s buckets: (691200, 14)


In [10]:
df_30s

,stock_id,time_id,bucket_start,log_ret_mean,log_ret_std,log_ret_sum,abs_volatility,realized_volatility,spread,imbalance,depth,depth_ratio,rv_rolling,mom_rolling
0,8382.0,12.0,0,-0.000072,0.000300,-0.002023,0.005951,0.001607,0.370714,0.102052,7107.0,2.281320,0.000486,-0.000097
1,8382.0,12.0,30,0.000015,0.000409,0.000440,0.008193,0.002206,0.382667,0.089951,17783.0,5.571606,0.000303,0.000132
2,8382.0,12.0,60,-0.000020,0.000258,-0.000586,0.005349,0.001391,0.304000,0.098040,7844.0,5.975772,0.000334,-0.000207
3,8382.0,12.0,90,0.000030,0.000182,0.000898,0.003616,0.000996,0.306500,0.159914,10877.0,6.003413,0.000235,0.000044
4,8382.0,12.0,120,0.000036,0.000211,0.001094,0.004162,0.001156,0.367000,-0.026375,9224.0,2.293206,0.000315,-0.000006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1382395,104919.0,1199.0,3450,0.000012,0.000066,0.000359,0.001406,0.000361,0.021000,-0.061605,155118.0,1.207049,0.000025,-0.000011
1382396,104919.0,1199.0,3480,-0.000010,0.000049,-0.000303,0.000965,0.000271,0.020333,0.148361,219742.0,1.880400,0.000012,0.000006
1382397,104919.0,1199.0,3510,0.000000,0.000037,0.000000,0.000855,0.000201,0.020667,-0.045085,189793.0,1.605021,0.000023,-0.000006
1382398,104919.0,1199.0,3540,0.000026,0.000046,0.000786,0.001117,0.000288,0.021667,0.144074,199447.0,1.835623,0.000018,0.000019


In [11]:
df_60s

,stock_id,time_id,bucket_start,log_ret_mean,log_ret_std,log_ret_sum,abs_volatility,realized_volatility,spread,imbalance,depth,depth_ratio,rv_rolling,mom_rolling
0,8382.0,12.0,0,-0.000027,0.000360,-0.001583,0.014144,0.002729,0.376897,0.095793,24890.0,3.983192,0.000303,0.000132
1,8382.0,12.0,60,0.000005,0.000223,0.000312,0.008965,0.001711,0.305250,0.128977,18721.0,5.989592,0.000235,0.000044
2,8382.0,12.0,120,0.000029,0.000187,0.001738,0.007604,0.001453,0.360833,0.008997,18513.0,3.119304,0.000103,0.000053
3,8382.0,12.0,180,-0.000008,0.000215,-0.000467,0.007326,0.001650,0.301833,0.031202,35827.0,4.447586,0.000012,0.000025
4,8382.0,12.0,240,0.000050,0.000189,0.002992,0.008347,0.001500,0.291500,-0.074440,41679.0,3.859149,0.000116,-0.000091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691195,104919.0,1199.0,3300,-0.000016,0.000059,-0.000978,0.002383,0.000473,0.021667,-0.104032,318752.0,0.956957,0.000012,-0.000006
691196,104919.0,1199.0,3360,-0.000012,0.000050,-0.000744,0.001985,0.000393,0.021833,0.017090,329727.0,1.365739,0.000026,0.000014
691197,104919.0,1199.0,3420,0.000006,0.000058,0.000386,0.002344,0.000449,0.021000,0.007757,340293.0,1.309447,0.000025,-0.000011
691198,104919.0,1199.0,3480,-0.000005,0.000044,-0.000303,0.001820,0.000337,0.020500,0.051638,409535.0,1.742711,0.000023,-0.000006


In [12]:
ref = pd.read_csv('time_id_reference.csv')

# 3. Merge on time_id to bring in the date
df_30s = df_60s.merge(ref, on='time_id', how='left')
df_30s['day'] = df_30s['date'].factorize()[0] + 1

In [14]:
df_30s

,stock_id,time_id,bucket_start,log_ret_mean,log_ret_std,log_ret_sum,abs_volatility,realized_volatility,spread,imbalance,depth,depth_ratio,rv_rolling,mom_rolling,date,time,day
0,8382.0,12.0,0,-0.000072,0.000300,-0.002023,0.005951,0.001607,0.370714,0.102052,7107.0,2.281320,0.000486,-0.000097,2021-01-05,11:00:00,1
1,8382.0,12.0,30,0.000015,0.000409,0.000440,0.008193,0.002206,0.382667,0.089951,17783.0,5.571606,0.000303,0.000132,2021-01-05,11:00:00,1
2,8382.0,12.0,60,-0.000020,0.000258,-0.000586,0.005349,0.001391,0.304000,0.098040,7844.0,5.975772,0.000334,-0.000207,2021-01-05,11:00:00,1
3,8382.0,12.0,90,0.000030,0.000182,0.000898,0.003616,0.000996,0.306500,0.159914,10877.0,6.003413,0.000235,0.000044,2021-01-05,11:00:00,1
4,8382.0,12.0,120,0.000036,0.000211,0.001094,0.004162,0.001156,0.367000,-0.026375,9224.0,2.293206,0.000315,-0.000006,2021-01-05,11:00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1382395,104919.0,1199.0,3450,0.000012,0.000066,0.000359,0.001406,0.000361,0.021000,-0.061605,155118.0,1.207049,0.000025,-0.000011,2021-10-07,16:00:00,192
1382396,104919.0,1199.0,3480,-0.000010,0.000049,-0.000303,0.000965,0.000271,0.020333,0.148361,219742.0,1.880400,0.000012,0.000006,2021-10-07,16:00:00,192
1382397,104919.0,1199.0,3510,0.000000,0.000037,0.000000,0.000855,0.000201,0.020667,-0.045085,189793.0,1.605021,0.000023,-0.000006,2021-10-07,16:00:00,192
1382398,104919.0,1199.0,3540,0.000026,0.000046,0.000786,0.001117,0.000288,0.021667,0.144074,199447.0,1.835623,0.000018,0.000019,2021-10-07,16:00:00,192


In [16]:
# 1. compute day-of-week as 1=Mon…5=Fri in a cycle
df_30s['day_of_week'] = ((df_30s['day'] - 1) % 5) + 1

# (optional) map to names
dow_map = {1:'Monday', 2:'Tuesday', 3:'Wednesday', 4:'Thursday', 5:'Friday'}
df_30s['day_name'] = df_30s['day_of_week'].map(dow_map)

# 2. compute week number: days 1–5 → week 1, 6–10 → week 2, etc.
df_30s['week'] = ((df_30s['day'] - 1) // 5) + 1

# if you just want the weeks column:
# df_10s['weeks'] = ((df_10s['day'] - 1) // 5) + 1
df_30s = df_30s[df_30s['week'] != 39].reset_index(drop=True)

df_30s


,stock_id,time_id,bucket_start,log_ret_mean,log_ret_std,log_ret_sum,abs_volatility,realized_volatility,spread,imbalance,depth,depth_ratio,rv_rolling,mom_rolling,date,time,day,day_of_week,day_name,week
0,8382.0,12.0,0,-7.225332e-05,0.000300,-0.002023,0.005951,0.001607,0.370714,0.102052,7107.0,2.281320,0.000486,-0.000097,2021-01-05,11:00:00,1,1,Monday,1
1,8382.0,12.0,30,1.467673e-05,0.000409,0.000440,0.008193,0.002206,0.382667,0.089951,17783.0,5.571606,0.000303,0.000132,2021-01-05,11:00:00,1,1,Monday,1
2,8382.0,12.0,60,-1.953187e-05,0.000258,-0.000586,0.005349,0.001391,0.304000,0.098040,7844.0,5.975772,0.000334,-0.000207,2021-01-05,11:00:00,1,1,Monday,1
3,8382.0,12.0,90,2.992879e-05,0.000182,0.000898,0.003616,0.000996,0.306500,0.159914,10877.0,6.003413,0.000235,0.000044,2021-01-05,11:00:00,1,1,Monday,1
4,8382.0,12.0,120,3.647908e-05,0.000211,0.001094,0.004162,0.001156,0.367000,-0.026375,9224.0,2.293206,0.000315,-0.000006,2021-01-05,11:00:00,1,1,Monday,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367995,104919.0,1187.0,3450,1.118553e-05,0.000047,0.000336,0.001035,0.000262,0.022333,0.267009,210074.0,2.020754,0.000054,0.000034,2021-10-05,16:00:00,190,5,Friday,38
1367996,104919.0,1187.0,3480,-1.724592e-05,0.000060,-0.000517,0.001356,0.000334,0.022333,0.004158,203621.0,1.432123,0.000027,0.000011,2021-10-05,16:00:00,190,5,Friday,38
1367997,104919.0,1187.0,3510,-4.662298e-07,0.000058,-0.000014,0.001049,0.000314,0.021333,-0.031581,176393.0,1.261690,0.000046,-0.000011,2021-10-05,16:00:00,190,5,Friday,38
1367998,104919.0,1187.0,3540,-5.595266e-06,0.000072,-0.000168,0.001539,0.000388,0.022000,0.185284,257750.0,1.853486,0.000064,0.000022,2021-10-05,16:00:00,190,5,Friday,38


In [19]:
df_30s.to_parquet("df_30s.parquet", index=False)


In [20]:
ref = pd.read_csv('time_id_reference.csv')

# 3. Merge on time_id to bring in the date
df_60s = df_60s.merge(ref, on='time_id', how='left')
df_60s['day'] = df_60s['date'].factorize()[0] + 1

In [21]:
# 1. compute day-of-week as 1=Mon…5=Fri in a cycle
df_60s['day_of_week'] = ((df_60s['day'] - 1) % 5) + 1

# (optional) map to names
dow_map = {1:'Monday', 2:'Tuesday', 3:'Wednesday', 4:'Thursday', 5:'Friday'}
df_60s['day_name'] = df_60s['day_of_week'].map(dow_map)

# 2. compute week number: days 1–5 → week 1, 6–10 → week 2, etc.
df_60s['week'] = ((df_60s['day'] - 1) // 5) + 1

# if you just want the weeks column:
# df_10s['weeks'] = ((df_10s['day'] - 1) // 5) + 1
df_60s = df_60s[df_60s['week'] != 39].reset_index(drop=True)

df_60s


,stock_id,time_id,bucket_start,log_ret_mean,log_ret_std,log_ret_sum,abs_volatility,realized_volatility,spread,imbalance,depth,depth_ratio,rv_rolling,mom_rolling,date,time,day,day_of_week,day_name,week
0,8382.0,12.0,0,-0.000027,0.000360,-0.001583,0.014144,0.002729,0.376897,0.095793,24890.0,3.983192,0.000303,0.000132,2021-01-05,11:00:00,1,1,Monday,1
1,8382.0,12.0,60,0.000005,0.000223,0.000312,0.008965,0.001711,0.305250,0.128977,18721.0,5.989592,0.000235,0.000044,2021-01-05,11:00:00,1,1,Monday,1
2,8382.0,12.0,120,0.000029,0.000187,0.001738,0.007604,0.001453,0.360833,0.008997,18513.0,3.119304,0.000103,0.000053,2021-01-05,11:00:00,1,1,Monday,1
3,8382.0,12.0,180,-0.000008,0.000215,-0.000467,0.007326,0.001650,0.301833,0.031202,35827.0,4.447586,0.000012,0.000025,2021-01-05,11:00:00,1,1,Monday,1
4,8382.0,12.0,240,0.000050,0.000189,0.002992,0.008347,0.001500,0.291500,-0.074440,41679.0,3.859149,0.000116,-0.000091,2021-01-05,11:00:00,1,1,Monday,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683995,104919.0,1187.0,3300,-0.000011,0.000061,-0.000685,0.002559,0.000473,0.021667,-0.040270,301254.0,1.174866,0.000070,-0.000014,2021-10-05,16:00:00,190,5,Friday,38
683996,104919.0,1187.0,3360,0.000003,0.000040,0.000154,0.001720,0.000309,0.020667,0.131692,329777.0,1.540810,0.000038,0.000011,2021-10-05,16:00:00,190,5,Friday,38
683997,104919.0,1187.0,3420,0.000007,0.000060,0.000419,0.002265,0.000468,0.021833,0.094141,416248.0,1.548123,0.000054,0.000034,2021-10-05,16:00:00,190,5,Friday,38
683998,104919.0,1187.0,3480,-0.000009,0.000059,-0.000531,0.002406,0.000459,0.021833,-0.013711,380014.0,1.346906,0.000046,-0.000011,2021-10-05,16:00:00,190,5,Friday,38


In [22]:
df_60s.to_parquet("df_60s.parquet", index=False)
